In [11]:
import torch
from transformers import BertForTokenClassification
from transformers import BertTokenizerFast

In [14]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-cased')
# text_tokenized = tokenizer(example, padding='max_length', max_length=512, truncation=True, return_tensors="pt")

In [15]:
def align_word_ids(texts):
    tokenized_inputs = tokenizer(texts, padding='max_length', max_length=512, truncation=True)

    word_ids = tokenized_inputs.word_ids()

    previous_word_idx = None
    label_ids = []

    for word_idx in word_ids:

        if word_idx is None:
            label_ids.append(-100)

        elif word_idx != previous_word_idx:
            try:
                label_ids.append(1)
            except:
                label_ids.append(-100)
        else:
            try:
                label_ids.append(1 if label_all_tokens else -100)
            except:
                label_ids.append(-100)
        previous_word_idx = word_idx

    return label_ids


def evaluate_one_text(model, sentence):
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:
        model = model.cuda()

    text = tokenizer(sentence, padding='max_length', max_length = 512, truncation=True, return_tensors="pt")

    mask = text['attention_mask'].to(device)
    input_id = text['input_ids'].to(device)
    label_ids = torch.Tensor(align_word_ids(sentence)).unsqueeze(0).to(device)

    logits = model(input_id, mask, None)
    logits_clean = logits[0][label_ids != -100]

    predictions = logits_clean.argmax(dim=1).tolist()
    prediction_label = [ids_to_labels[i] for i in predictions]
    print(sentence)
    print(prediction_label)
            

In [21]:
saved_model = torch.load(r'C:\Users\Admin\Documents\University\Current Class\IR and TM\DaVinciCodeTheTrackOfRobertLangdon\models\small-finetuned-NER-bert-case')
evaluate_one_text(saved_model, 'Bill Gates is the founder of Microsoft')

AttributeError: Can't get attribute 'BertModel' on <module '__main__'>